In [ ]:
DATA_DIR = "./data"
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
!mkdir -p $DATA_DIR
!test ! -d $DATA_DIR/train \
    && wget -O $DATA_DIR/train.tar https://people.eecs.berkeley.edu/~hendrycks/streethazards_train.tar \
    && tar -xf $DATA_DIR/train.tar -C $DATA_DIR \
    && rm -r $DATA_DIR/train.tar \
    && mv $DATA_DIR/train $DATA_DIR/streethazards_train
!test ! -d $DATA_DIR/test \
    && wget -O $DATA_DIR/test.tar https://people.eecs.berkeley.edu/~hendrycks/streethazards_test.tar \
    && tar -xf $DATA_DIR/test.tar -C $DATA_DIR \
    && rm -r $DATA_DIR/test.tar\
    && mv $DATA_DIR/test $DATA_DIR/streethazards_test

In [ ]:
!pip install -U segmentation-models-pytorch

In [ ]:
import numpy as np
import os
from enum import IntEnum
import torch
from torch import Tensor
import torch.nn as nn
import segmentation_models_pytorch as smp
from typing import Optional, Callable, Union, Tuple, Dict, List
import json
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms
from torchvision.transforms import v2
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
known_classes = 13

In [ ]:
"""
Source: https://github.com/hendrycks/anomaly-seg/issues/15#issuecomment-890300278
"""
COLORS = np.array([
[  0,   0,   0], # // unlabeled     =   0, black
[ 70,  70,  70], # // building      =   1, gray
[190, 153, 153], # // fence         =   2, pinkish
[250, 170, 160], # // other         =   3, pink
[220,  20,  60], # // pedestrian    =   4, 
[153, 153, 153], # // pole          =   5,
[157, 234,  50], # // road line     =   6, 
[128,  64, 128], # // road          =   7,
[244,  35, 232], # // sidewalk      =   8,
[107, 142,  35], # // vegetation    =   9, 
[  0,   0, 142], # // car           =  10,
[102, 102, 156], # // wall          =  11, 
[220, 220,   0], # // traffic sign  =  12,
[ 60, 250, 240], # // anomaly       =  13,
]) 


def visualize_annotation(annotation_img: np.ndarray | torch.Tensor, ax=None, title= None) -> None:
    """
    Visualize a segmentation annotation using a predefined color palette.

    Args:
        annotation_img (np.ndarray | torch.Tensor): 2D array with class indices.
        ax (matplotlib.axes.Axes, optional): Axis to plot on. If None, uses current axis.
    """
    if ax is None: ax = plt.gca()
    annotation_img = np.asarray(annotation_img)
    img_new = np.zeros((*annotation_img.shape, 3))

    for index, color in enumerate(COLORS):
        img_new[annotation_img == index] = color

    ax.imshow(img_new / 255.0)
    if title:
        ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

def visualize_scene(image: np.ndarray | torch.Tensor, ax=None, title= None) -> None:
    """
    Visualize a raw RGB scene image.

    Args:
        image (np.ndarray | torch.Tensor): Image tensor or array in [C, H, W] or [H, W, C] format.
        ax (matplotlib.axes.Axes, optional): Axis to plot on. If None, uses current axis.
    """
    if ax is None: ax = plt.gca()
    image = np.asarray(image)
    ax.imshow(np.moveaxis(image, 0, -1))
    if title:
        ax.set_title(title)
    ax.set_xticks([])
    ax.set_yticks([])

    
#path to streethazards dataset
train_odgt_file = f"{DATA_DIR}/streethazards_train/train.odgt"
val_odgt_file = f"{DATA_DIR}/streethazards_train/validation.odgt"
test_odgt_file = f"{DATA_DIR}/streethazards_test/test.odgt"

COMPUTE_MEAN_STD = False

In [ ]:
class StreetHazardsDataset(Dataset):
    """
    A custom PyTorch Dataset for the StreetHazards inliner dataset.

    This dataset reads image and segmentation label paths from a `.odgt` file,
    applies optional resizing and spatial transformations, and returns
    dictionary-style samples with normalized image tensors and label tensors.

    Args:
        odgt_file (str): Path to the `.odgt` file containing image and label metadata.
        image_resize (Tuple[int, int], optional): Target size to resize images and labels. 
        spatial_transforms (Callable, optional): Optional transformation function applied to both images and labels.
        mean_std (Tuple[List[float], List[float]], optional): Mean and standard deviation for image normalization.
        
    """
    def __init__(
        self,
        odgt_file: str,
        image_resize: Tuple[int, int] = (512, 896),
        spatial_transforms: Optional[Callable] = None,
        mean_std: Tuple[List[float], List[float]] = None
    ):

        self.spatial_transforms = spatial_transforms
        self.mean_std = mean_std
        self.image_resize = image_resize

        with open(odgt_file, "r") as f:
            odgt_data = json.load(f)
        

        self.paths = [
            {
                "image": os.path.join(Path(odgt_file).parent, data["fpath_img"]),
                "labels": os.path.join(Path(odgt_file).parent, data["fpath_segm"]),
            }
            for data in odgt_data 
        ]
    
    def __len__(self) -> int:
        return len(self.paths)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:

        image = Image.open(self.paths[idx]["image"]).convert("RGB")
        labels = Image.open(self.paths[idx]["labels"])

        if self.image_resize:
            image = transforms.Resize(self.image_resize, transforms.InterpolationMode.BILINEAR)(image)
            labels = transforms.Resize(self.image_resize, transforms.InterpolationMode.NEAREST)(labels)
            
        if self.spatial_transforms:
            image, labels  = self.spatial_transforms(image, labels)         

        #to_tensor
        image = transforms.ToTensor()(image)
        labels = torch.as_tensor(transforms.functional.pil_to_tensor(labels), dtype=torch.int64) - 1
        
        labels = labels.squeeze(0)
        
        if self.mean_std:
            image = transforms.Normalize(mean = self.mean_std[0], std = self.mean_std[1])(image)

        return {'image' : image, 'labels' : labels}

In [ ]:
def create_one_hot_prototypes_torch(num_known_classes: int, t_value: float = 3.0, device: str = 'cpu') -> torch.Tensor:
    """
    Generates one-hot prototypes as a PyTorch tensor for a given number of known classes.
    Each prototype is a vector where only the element corresponding
    to its class index has the 't_value', and all other elements are 0.

    Args:
        num_known_classes (int): The total number of known (in-distribution) classes.
                                 This also determines the dimensionality of each prototype vector.
        t_value (float): The non-zero value at the class's specific index in the prototype.
                         As specified in the paper, this is often 3.0.
        device (str): The device on which to create the tensor ('cpu' or 'cuda').

    Returns:
        torch.Tensor: A 2D PyTorch tensor where each row is a prototype vector.
                      The shape will be (num_known_classes, num_known_classes).
    """
    if not isinstance(num_known_classes, int) or num_known_classes <= 0:
        raise ValueError("num_known_classes must be a positive integer.")
    if not isinstance(t_value, (int, float)):
        raise ValueError("t_value must be a numeric type.")
    if device not in ['cpu', 'cuda']:
        raise ValueError("device must be 'cpu' or 'cuda'.")

    # Create a tensor of zeros
    prototypes = torch.zeros((num_known_classes, num_known_classes), dtype=torch.float32, device=device)

    # Fill the diagonal with t_value to create one-hot prototypes
    for i in range(num_known_classes):
        prototypes[i, i] = t_value
        
    # An even more concise way using torch.eye (Identity matrix)
    # prototypes = torch.eye(num_known_classes, dtype=torch.float32, device=device) * t_value

    return prototypes

Prototype  = create_one_hot_prototypes_torch(known_classes)

In [ ]:

class DMLNetFeatureExtractor(torch.nn.Module):
    def __init__(self, encoder_name, encoder_weights, num_feature_channels, activation):
        super().__init__()
        

        self.model = smp.DeepLabV3Plus(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            classes=num_feature_channels, # This sets the output channels of the segmentation_head if kept
            activation=activation # Usually 'None' for the main head, but for features it might not matter directly
        )
        

        # Option 2 is safer and more robust.
        # First, we disable the original segmentation head as you did.
        self.original_segmentation_head = self.model.segmentation_head # Store it if needed
        self.model.segmentation_head = torch.nn.Identity() # Remove the final head

        # --- CORRECTION START ---
        # To get the decoder's actual output channels, we need a dummy forward pass
        # through just the encoder and decoder.
        
        # Temporarily detach the module to make a dummy pass if needed,
        # but in __init__, we can usually just do a conceptual forward.
        # However, to be absolutely safe and get the runtime channel count:
        
        # Create a dummy input to trace the decoder output channels
        # Assuming typical RGB input (3 channels) and arbitrary spatial dimensions
        dummy_input = torch.randn(2, 3, 256, 256) 
        
        # Pass through encoder
        encoder_features_dummy = self.model.encoder(dummy_input)
        
        # Pass through decoder to get its output channels
        decoder_output_dummy = self.model.decoder(encoder_features_dummy)
        
        # Extract the channel dimension from the dummy output
        decoder_actual_out_channels = decoder_output_dummy.shape[1]
        # --- CORRECTION END ---

        # Add a 1x1 convolution to project the decoder's output to the desired num_feature_channels.
        self.feature_projection = torch.nn.Conv2d(
            in_channels=decoder_actual_out_channels,
            out_channels=num_feature_channels,
            kernel_size=1,
            stride=1,
            padding=0
        )
        
    def forward(self, x):
        input_spatial_size = x.shape[2:]
        # The encoder outputs a list of feature maps at different resolutions
        encoder_features = self.model.encoder(x)
        
        # The decoder takes these features and produces a high-resolution feature map.
        # This output will typically have the same spatial dimensions as the input 'x'
        # (due to DeepLabV3+ decoder's upsampling) but with its default channel count.
        decoder_output = self.model.decoder(encoder_features)
        
        # Project the decoder's output to the desired number of feature channels
        projected_features = self.feature_projection(decoder_output)

        final_features = torch.nn.functional.interpolate(
            projected_features, 
            size=input_spatial_size, 
            mode='bilinear', 
            align_corners=False # Set to True for pixel alignment if needed, but False is common
        )
        
        # These `final_features` are your f(X; θf)i,j with num_feature_channels.
        return final_features

In [ ]:
class DiscriminativeCrossEntropyLoss(nn.Module):
    def __init__(self, prototypes: torch.Tensor,lambda_weight):
        super().__init__()
       
        self.prototypes = prototypes.to(device)
        self.lambda_weight = lambda_weight

    def forward(self, pixel_features: torch.Tensor, target_labels: torch.Tensor):
        # pixel_features: (B, D, H, W)
        # target_labels: (B, H, W)

        # Reshape pixel_features for easier broadcasting with prototypes
        # (B, D, H, W) -> (B, H, W, D)
        pixel_features_reshaped = pixel_features.permute(0, 2, 3, 1) # (B, H, W, D)

        # Expand target_labels to match the feature dimension for indexing prototypes
        # (B, H, W) -> (B, H, W, D)
        prototypes_target = self.prototypes[target_labels] # This should now be (B, H, W, D)

        # --- Numerator (Attractive Force) ---
        # ||f(X; θf)i,j – mY i,j ||²
        # (B, H, W, D) - (B, H, W, D) -> (B, H, W, D)
        difference_numerator = pixel_features_reshaped - prototypes_target
        # (B, H, W, D) -> (B, H, W)
        squared_diff_numerator = torch.sum(difference_numerator.pow(2), dim=-1)
        exp_squared_diff_numerator = torch.exp(-squared_diff_numerator)

        # --- Denominator (Repulsive Force) ---
        # Σk=1 to N ( exp(-||f(X; θf)i,j – mk ||²) )
        # To compute this, we need to calculate the squared difference for ALL prototypes
        # and then sum their exponentials.

        # Expand pixel_features to compare with all prototypes: (B, H, W, 1, D)
        pixel_features_expanded = pixel_features_reshaped.unsqueeze(-2) # (B, H, W, 1, D)

        # Expand prototypes to compare with all pixels: (1, 1, 1, N, D)
        # self.prototypes has shape (N, D)
        prototypes_expanded = self.prototypes.unsqueeze(0).unsqueeze(0).unsqueeze(0) # (1, 1, 1, N, D)

        # Calculate difference between each pixel feature and ALL prototypes
        # (B, H, W, 1, D) - (1, 1, 1, N, D) -> (B, H, W, N, D)
        all_prototypes_differences = pixel_features_expanded - prototypes_expanded

        # Square and sum across the feature dimension (D)
        # (B, H, W, N, D) -> (B, H, W, N)
        squared_diff_all_prototypes = torch.sum(all_prototypes_differences.pow(2), dim=-1)

        # Exponentiate
        # (B, H, W, N)
        exp_squared_diff_all_prototypes = torch.exp(-squared_diff_all_prototypes)

        # Sum across the prototype dimension (N) to get the denominator
        # (B, H, W, N) -> (B, H, W)
        denominator = torch.sum(exp_squared_diff_all_prototypes, dim=-1)

        # --- Calculate Pt(Xi,j) (Equation 2) ---
        # (B, H, W) / (B, H, W) -> (B, H, W)
        pt_values = exp_squared_diff_numerator / (denominator + 1e-8) # Add small epsilon for stability

        # --- Calculate LDCE (Equation 3) ---
        # LDCE = -log(pt_values) for target classes
        # This requires masking based on target_labels or using the pt_values directly
        # The equation shows sum over i,j of -log(numerator/denominator) where numerator corresponds to Y_i,j

        # Assuming Y_i,j is 1 for the target class at that pixel and 0 otherwise.
        # This is essentially -log(Pt(Xi,j)) for the correct class, summed over all pixels.
        ldce_loss = -torch.log(pt_values + 1e-8) # Add small epsilon for stability
        ldce_loss = torch.sum(ldce_loss) # Or torch.sum() depending on how you want to aggregate

        Lvl_loss = torch.sum(difference_numerator.pow(2))
        
        # Check for NaNs/Infs
        if torch.isnan(pixel_features_reshaped).any():
            print("NaN found in pixel_features_reshaped!")
        if torch.isinf(pixel_features_reshaped).any():
            print("Inf found in pixel_features_reshaped!")
        
        
        return ldce_loss + self.lambda_weight * Lvl_loss
       

In [ ]:
shape_resize = (512, 896)

spatial_transforms = transforms.v2.Compose([
    transforms.v2.RandomHorizontalFlip(),
])

train_dataset = StreetHazardsDataset(
    odgt_file= train_odgt_file
)

val_dataset = StreetHazardsDataset(
    odgt_file= val_odgt_file
)

test_dataset = StreetHazardsDataset(
    odgt_file= test_odgt_file
)

train_dl = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_dl = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)
test_dl = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Esempio di utilizzo:
encoder_name = "efficientnet-b2"
encoder_weights = "imagenet"
t_value = 3.0
epochs = 100

# Inizializza il feature extractor
feature_extractor = DMLNetFeatureExtractor(
    encoder_name=encoder_name,
    encoder_weights=encoder_weights,
    num_feature_channels=known_classes,
    activation=None
).to(device)
model_optimizer = torch.optim.Adam(feature_extractor.parameters(), lr=0.02)

def train(num_epochs,model,train_loader,lambda_weight) -> None:
        
        for epoch in tqdm(range(num_epochs), desc="Epoch"):
            
            model.train()

            losses = []

            for batch in train_loader: 

                    
                imgs = batch['image'].to(device)
                labels = batch['labels'].to(device)
                
                logits = model(imgs)
                
                lossClass = DiscriminativeCrossEntropyLoss(Prototype,lambda_weight)

                
                loss = lossClass(logits,labels)
                            
                losses.append(loss.item())
                
                model_optimizer.zero_grad()
                loss.backward()
                model_optimizer.step()
            
                del loss
                

            l = sum(losses) / len(losses)

            print(f"Epoch {epoch + 1} Loss {l}", end = ' ')

In [ ]:
#loaded_model = torch.load("/kaggle/input/dml/pytorch/default/10/model_weights10E_resnet34.pth",weights_only=False)
#feature_extractor = loaded_model
#print("model loaded")

In [ ]:
train(epochs,feature_extractor,train_dl,0.1)

In [ ]:
torch.save(feature_extractor, "model_weights100E_efficientnet-b4.pth")

In [ ]:
#loaded_model = torch.load("/kaggle/input/dml/pytorch/default/4/model.weights.pth",weights_only=False)
#loaded_model.eval()
#feature_extractor = loaded_model

In [ ]:
def get_probability(logits, Prototype, device): # Passa Prototype e device come argomenti
    # logits: (B, D, H, W)
    # Prototype: (N_CLASSES, D)  -> N_CLASSES dovrebbe essere 12 nel tuo caso
    
    # Reshape logits for easier broadcasting with prototypes
    # (B, D, H, W) -> (B, H, W, D)
    pixel_features_reshaped = logits.permute(0, 2, 3, 1) # (B, H, W, D)
    
    B, H, W, D = pixel_features_reshaped.shape
    N_CLASSES = Prototype.shape[0] # Numero di classi/prototipi, dovrebbe essere 12
    
    # Inizializza il tensore 'probability' con zeri. Questo è il tensore di output finale.
    probability = torch.zeros(B, H, W, N_CLASSES, device=device)

    # Sposta Prototype sulla device una sola volta, se non è già lì
    Prototypes_on_device = Prototype.to(device)

    # --- Pre-calcolo delle differenze e delle somme per il denominatore (più efficiente) ---
    # Per il denominatore, abbiamo bisogno di calcolare le distanze di ogni pixel_feature da *tutti* i prototipi.
    # Questo può essere fatto in modo vettoriale, senza un loop sui prototipi per questa parte.

    # Expand pixel_features_reshaped to compare with all prototypes: (B, H, W, 1, D)
    pixel_features_expanded_for_denom = pixel_features_reshaped.unsqueeze(-2) 
    
    # Expand Prototypes_on_device to compare with all pixels: (1, 1, 1, N_CLASSES, D)
    # (N_CLASSES, D) -> (1, 1, 1, N_CLASSES, D)
    prototypes_expanded_for_denom = Prototypes_on_device.unsqueeze(0).unsqueeze(0).unsqueeze(0) 

    # Calculate difference between each pixel feature and ALL prototypes
    # (B, H, W, 1, D) - (1, 1, 1, N_CLASSES, D) -> (B, H, W, N_CLASSES, D)
    all_prototypes_differences = pixel_features_expanded_for_denom - prototypes_expanded_for_denom
    
    # Square and sum across the feature dimension (D)
    # (B, H, W, N_CLASSES, D) -> (B, H, W, N_CLASSES)
    squared_diff_all_prototypes = torch.sum(all_prototypes_differences.pow(2), dim=-1)
    
    # Exponentiate
    # (B, H, W, N_CLASSES)
    exp_squared_diff_all_prototypes = torch.exp(-squared_diff_all_prototypes)
    
    # Sum across the prototype dimension (N_CLASSES) to get the denominator for *all* classes
    # (B, H, W, N_CLASSES) -> (B, H, W)
    denominator = torch.sum(exp_squared_diff_all_prototypes, dim=-1, keepdim=True) + 1e-8 # keepdim per broadcasting

    # Ora loop per calcolare il numeratore e la probabilità per ogni classe 't'
    for t in range(N_CLASSES):
        # prototypes_t è il prototipo specifico per la classe 't': (D,)
        prototypes_t = Prototypes_on_device[t] 
    
        # --- Numerator (Attractive Force) ---
        # ||f(X; θf)i,j – m_t ||²  (m_t è il prototipo per la classe t)
        # (B, H, W, D) - (D) -> (B, H, W, D) (broadcasting)
        difference_numerator = pixel_features_reshaped - prototypes_t
        
        # (B, H, W, D) -> (B, H, W)
        squared_diff_numerator = torch.sum(difference_numerator.pow(2), dim=-1)
        exp_squared_diff_numerator = torch.exp(-squared_diff_numerator)
    
        # --- Calculate Pt(Xi,j) (Equation 2) ---
        # (B, H, W) / (B, H, W) -> (B, H, W)
        pt_values = exp_squared_diff_numerator / denominator.squeeze(-1) # Rimuovi la dimensione aggiunta per il broadcasting
        
        # Assegna i valori di probabilità per la classe 't'
        probability[:,:,:,t] = pt_values
    
    # Restituisci il tensore completo delle probabilità dopo il ciclo
    return probability

In [ ]:
torch.cuda.memory._record_memory_history()

In [ ]:
feature_extractor.eval()
with torch.no_grad():
    for batch in test_dl:
    
        imgs = batch['image'].to(device)
        labels = batch['labels'].to(device)
                    
        logits = feature_extractor(imgs)
        probability_vector = get_probability(logits,Prototype,device)
    
        fig, axs = plt.subplots(1, 3, figsize=(13, 13))
        idx = 3
        visualize_scene(imgs.cpu()[5], axs[0])
        visualize_annotation(torch.argmax(probability_vector,axis=3).cpu().detach().numpy()[5], axs[1])
        visualize_annotation(labels.cpu().detach().numpy()[5],axs[2])
        
        break
    
    

In [ ]:
def MMSP(probability_map):
    MMSP_map = 1 - torch.max(probability_map, dim=3)[0] 
    return MMSP_map

In [ ]:
def Probability_EDS(pixel_feature):
    pixel_features_reshaped = pixel_feature.permute(0, 2, 3, 1) 

    S_xy = torch.zeros(pixel_features_reshaped.shape[0], pixel_features_reshaped.shape[1], pixel_features_reshaped.shape[2], device=pixel_feature.device)
    
    for t in range(known_classes):
        # Assicurati che Prototype[t] abbia la forma corretta per il broadcasting
        # Se Prototype è (known_classes, D), allora .view(1, 1, 1, -1) lo rende (1, 1, 1, D)
        prototypes_t = Prototype[t].to(device).view(1, 1, 1, -1)
        
        difference_numerator = pixel_features_reshaped - prototypes_t
        
        # Somma dei quadrati lungo la dimensione D (ultima dimensione)
        squared_diff_numerator = torch.sum(difference_numerator**2, dim=3)
        S_xy = S_xy + squared_diff_numerator

    MaxS = torch.max(S_xy) # torch.max su un tensore singolo restituisce un tensore
    P_EDS = 1 - S_xy / (MaxS + 1e-8) # Aggiungi epsilon per stabilità numerica
    
    return P_EDS

        
    

In [ ]:
with torch.no_grad():
    for batch in test_dl:
    
        imgs = batch['image'].to(device)
        labels = batch['labels'].to(device)
                    
        logits = feature_extractor(imgs)
        probability_vector = get_probability(logits,Prototype,device)
    
        fig, axs = plt.subplots(1, 2, figsize=(10, 12))
        idx = 2
        
        visualize_scene(np.rot90(MMSP(probability_vector)[5].cpu().detach().numpy()), axs[0])
        visualize_scene(np.rot90(Probability_EDS(logits)[5].cpu().detach().numpy()), axs[1])
        
        break
    

In [ ]:
def Anomaly_probability(MMSP_map, P_EDS_map, gamma, beta):

    alpha = 1 / (1 + torch.exp(-beta * (P_EDS_map - gamma)))

    Anomaly_p = alpha * P_EDS_map + (1 - alpha) * MMSP_map
    return Anomaly_p

def Final_map(logits, lambda_value, gamma, beta):

    probability_vector = get_probability(logits, Prototype, device) 
    vector_mmsp = MMSP(probability_vector) 
    vector_eds = Probability_EDS(logits) 
    Anomaly_map = Anomaly_probability(vector_mmsp, vector_eds, gamma, beta) # (B, H, W)
    segmentation_map = torch.argmax(probability_vector, dim=3) # (B, H, W)
    ood_label = 13 
    final_map = torch.where(Anomaly_map <= lambda_value, segmentation_map, torch.full_like(segmentation_map, ood_label))

    return final_map.cpu().numpy(), Anomaly_map.cpu().numpy()
    

In [ ]:
lambda_value = 0.55
with torch.no_grad():
    for batch in test_dl:
    
        imgs = batch['image'].to(device)
        labels = batch['labels'].to(device)
    
        fig, axs = plt.subplots(1, 3, figsize=(10, 12))
        idx = 2

        visualize_annotation(Final_map(logits,lambda_value,0.5,20)[0][5], axs[0])
        visualize_scene(Final_map(logits,lambda_value,0.5,20)[1][5], axs[1])
        visualize_annotation(labels.cpu().detach().numpy()[5],axs[2])
        
        break

In [ ]:
torch.cuda.memory._dump_snapshot("my_snapshot.pickle")